# imports

In [18]:
import requests
from bs4 import BeautifulSoup
import bs4
import re
import pandas as pd

In [19]:
from random import randint


In [20]:
import time

In [103]:
import dask
from dask.diagnostics import ProgressBar
import multiprocessing

from tqdm import tqdm_notebook as tqdm


In [83]:
import pickle

# main funcs

In [84]:
def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

In [85]:
def url_generator():
    number =  random_with_N_digits(6)
    url = "https://www.carvana.com/vehicle/1" + str(number)
    return url

In [86]:
def request_text(url):
    hack = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

    r = requests.get(url, timeout = 30, headers = hack)
    r.raise_for_status()
    r.encoding = r.apparent_encoding
    return r.text
    

In [87]:
@dask.delayed
def find_gap(html_text):
    try:
        iterator = re.finditer(r'\bSave\b\s\$<!-- -->\d*\,\d+<!--\s-->',html_text)
        locations = []

        for match in iterator:
            locations.append((match.span()[0], match.span()[1]))
    
        for pair in locations:
            narrow_text = html_text[pair[0]:pair[1]]

        text = re.findall('>\d*\,\d+<',narrow_text)
        text = text[0][1:-1]
        text = text.replace(',',"")
        number = int(text)
        return number
    except:
        return None

In [88]:
@dask.delayed
def find_model(html_text):
    try:
        soup = bs4.BeautifulSoup(html_text, 'html.parser')
        title_lst= soup.find_all('title')
        find_lst= re.findall('[\w|\s]*\|',title_lst[0].text)
        title_lst[0].text
        return find_lst[0][:-1]
    except:
        return None

In [89]:
@dask.delayed
def find_price(html_text):
    try:
        lst = re.findall(',"price":\d\d\d\d0', html_text)
        return int(lst[0][-5:])
    except:
        return None

In [90]:
def findnth(haystack, needle, n):
    parts= haystack.split(needle, n+1)
    if len(parts)<=n+1:
        return -1
    return len(haystack)-len(parts[-1])-len(needle)

@dask.delayed
def find_status(html_text):
    try:
        lst = re.findall('lockType":"\w+","saleStatus":',html_text)
        text = lst[0][findnth(lst[0],'"',1)+1:findnth(lst[0],'"',2)]
        return text
    except:
        return None
    
    

# Implement

In [100]:
#### WARNING: do not run me!!!!!!!!!!!!! 
good_cars = []

In [104]:
def robot(max_run = 1000):
    t = 0
    for _ in tqdm(range(max_run), total=max_run):
        
        ## uncomment this if you are very noble
        #time.sleep(2)
        
        url = url_generator()
        try:
            html_text = request_text(url)
        except:
            #print("This URL is invalid")
            pass
        
        tasks = [find_gap(html_text), find_model(html_text), find_price(html_text), find_status(html_text)]
        
        lst = dask.compute(tasks, num_workers=multiprocessing.cpu_count())

        good_cars.append((url,lst[0][1],lst[0][0], lst[0][2], lst[0][3]))
        #print("finished the %d Web" % t)
        t += 1
        
        if t%(max_run//10) == 0:
            df = pd.DataFrame(good_cars, columns=['url','model','lower_than_kkb','price','status'])
            df['status'] = df['status'].map({'None': "Available", 'Purchase': "Non-available"})
            df.to_csv("partial_sav{}.csv".format(t//100))
    
    
    

In [ ]:
robot()

<ipython-input-104-b564b64a1577>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _ in tqdm(range(max_run), total=max_run):


[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100